# LyricistAI

## Sequence, Sequential?

* Sequence data : 나열된 데이터(비슷한 특징을 가지거나 같은 속성이여야 하는 것은 아니다.)
* 다만 인공지능에서는 예측을 위해서 data들이 서로 유사한 특징을 가져야한다. 이러한 데이터를 Sequential Data라 한다.
* AI가 문법을 익히기는 힘드니 통계적인 방법을 사용한다.

## 문장 만들기

### 문장 연결

* 수많은 글을 읽게 함으로써 다음 단어를 예측을 잘 할 수 있게 한다.
* start 를 주고 나온 결과를 다음 뉴런의 input으로 사용을 반복하고 end가 나옴 이러한 순환적 특징 때문에 순환 신경망(RNN)이라한다.

In [1]:
sentence = " 나는 밥을 먹었다 "

source_sentence = "<start>" + sentence
target_sentence = sentence + "<end>"

print("Source 문장:", source_sentence)
print("Target 문장:", target_sentence)

Source 문장: <start> 나는 밥을 먹었다 
Target 문장:  나는 밥을 먹었다 <end>


### 언어 모델

* 단어 다음에 무슨 단어가 나올지 예측하는 확률 모델을 언어 모델이라 한다.
* P(w<sub>n</sub>​∣w<sub>1</sub> ,...,w<sub>n−1</sub>; θ)

## 데이터 다듬기

### 데이터 불러오기

* 파일을 읽기로 열고 라인 단위로 끊어서 list 형태로 읽어옵니다.
* 앞에서부터 10라인만 화면에 출력해 볼까요?

In [2]:
import os, re 
import numpy as np
import tensorflow as tf

# file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'
file_path = 'data/shakespeare.txt'

with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력해 볼까요?
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


### 데이터 전처리 : 공백과 화자 제거

* 여기서 공백인 문장과 화자가 표시된 문장을 삭제해주고 싶습니다.
* 길이가 0인 문장과 문장의 끝이 : 인 문장은 건너뜁니다.
* 그리고 문장을 10개만 확인합니다.

In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0:
        continue
    if sentence[-1] == ":":
        continue
    if idx > 9:
        break
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


토근화(Tokenize) : 문장을 일정한 기준으로 쪼개서 단어로 만드는 것

### 데이터 전처리 함수 설정 - 단어

1. 소문자로 바꾸고(lower) 양쪽 공백을 지웁니다(strip).
2. 특수 문자 양쪽에 공백을 넣습니다.
3. 여러 개의 공백을 하나로 만듭니다.
4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다.
5. 다시 양쪽 공백을 지웁니다.
6. 문장의 시작에는 <start>, 끝에는 <end>를 추가합니다.

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


### 데이터 전처리 함수 적용

* 문장의 길이가 0이거나 문자의 마지막이 : 인 경우는 제외합니다.
* 문장에 전처리 함수를 적용하고 저장합니다.

In [5]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0:
        continue
    if sentence[-1] == ":":
        continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

### Tokenizer 함수 설정

* 자연어 처리용 모듈 : tf.keras.preprocessing.text.Tokenizer
* 벡터화(vectorize) : 정제된 데이터를 토큰화 -> 단어 사전 생성 -> 데이터를 숫자로 변환
* tensor : 숫자로 변환된 데이터

링크 참조  
* [Tokenizer](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer) : 토큰화 관련 클래스
* [pad_sequences](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences) : 시퀀스를 같은 길이가 되도록 pad 합니다.

Tokenizer args
* num_words : 저장할 단어의 개수
* filters : 필터링 될 문자열(제거할 구두점 같은 것들), default 는 '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n' 이다.
* lower : 텍스트를 소문자로 만들 것인가? - boolean
* split : 단어를 나눌 기준(스페이스바 단위로 나눈다고 할때 ' ' 넣기)
* char_level : True 면 모든 캐릭터가 token화가 된다.
* oov_token : 적용하면 단어 index가 생기고 단어 사전에 저장되지 않는 값들을 이것으로 바꾼다(text_to_sequence call 동안)
* analyzer : 함수, 텍스트를 나눌 특정 분석. 기본 분석은 text_to_word_sequence 이다.  

Tokenizer method
* fit_on_sequences : sequence 에 따라 사전 최신화
* fit_on_texts : texts 에 따라 사전 최신화
* text_to_sequences : texts 를 정수 시퀀스로 변환

pad_sequences args
* sequence : 변환할 시퀀스
* maxlen : 최대 길이
* dtype : 데이터 타입(default : int32)
* padding : pre or past(pre: 앞에서부터 채움, past: 뒤에서부터 채움, pre 패딩이 더 좋다는 중론, default = pre)
* truncating : pre or past(maxlen보다 클 경우 앞부분을 지울지 뒷부분을 지울지 결정, default = pre)
* value : padding시 넣는 값(default : 0)

In [6]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras.preprocessing.text.Tokenizer object at 0x7fdf38791690>


In [7]:
tensor[:3]

array([[   2,  143,   40,  933,  140,  591,    4,  124,   24,  110,    5,
           3,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   2,  110,    4,  110,    5,    3,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   2,   11,   50,   43, 1201,  316,    9,  201,   74,    9, 3034,
          15,    3,    0,    0,    0,    0,    0,    0,    0,    0]],
      dtype=int32)

In [8]:
for idx in tokenizer.index_word.values():
    if idx == 0:
        print('Find 0!')

In [9]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


* 단어사전의 key = 1 에 사전에 들어가지 못한 단어들이 들어가며 value 는 <'unk'> 입니다.
* 사이즈에 맞춰서 뒤에 0이 생기지만 결국 마지막은 3(<'end'>)이며 시작은 2(<'start'>)입니다.
* 그렇기에 마지막 값은 <'pad'>일 가능성이 높습니다.

### 소스 및 타겟 문장 생성

In [10]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]

# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]

print(raw_corpus[:9])
print(src_input[0])
print(tgt_input[0])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']
[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


소스 문장은 맨뒤를 짜름, 모델에서 계산할 때 end 나오면 바로 끝나게 = 0을 건너뛰므로 무조건 맨뒤만 짜름  
만약 가장 긴 문장이라서 마지막 값이 end 라도 end도 결국 원래 setence가 아니기에 그냥 삭제  
타겟 문장은 정답이므로 맨앞을 짜름

### Dataset 사용

[tensorflow Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)

In [ ]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

* 소스 문장의 크기만큼 버퍼 사이즈를 잡는다.
* buffer : 전송하는 동안 그 데이터를 보관하는 메모리 영역
* batch size : 일괄처리량
* step
* 단어장의 전체 크기 : 원래 크기 + <'pad'>인 0 = 원래 크기 + 1